#Exercise 4

For this exercise, we simulate Conway's Game of Life and compute entropy.

We use the following packages:

NumPy -> allows us to use some math operations and NumPy arrays

Plotly.express -> allows us to create plots in a single line

SciPy -> allows for easy calculation of the number of alive neighbours for each cell

In [ ]:
import numpy as np
import plotly.express as px
from scipy.ndimage import convolve

In [ ]:
x = np.random.randint(0,2,size=(50,100))
fig = px.imshow(x)
fig.show()


In [ ]:
def step(x):
    # The cells with 1 are the ones being considered for calculation
    kernel = np.array([[1,1,1], [1,0,1], [1,1,1]])
    # This determines the number of cells surrounding the center cell that contain a 1 for all cells in x
    # neighbours[2][4] = n, where n is the number of 1s surrounding the cell (2, 4)
    neighbours = convolve(x, kernel)
    # To store the changes
    diff = np.zeros_like(neighbours)
    # The following logic is cool!
    # This goes through neighbours and x (both 2d arrays) and wherever both conditions are true diff is modified at that cell with the indicated value
    diff[(neighbours < 2) & (x == 1)] = -1 # dies of loneliness
    diff[(neighbours > 3) & (x == 1)] = -1 # dies due to overcrowding
    diff[(neighbours == 3) & (x == 0)] = +1 # becomes alive
    return x + diff

l = []
for _ in range(50):
    l += [x]
    x = step(x)
    
fig = px.imshow(np.array(l), animation_frame=0)
fig.show()

In [ ]:
# The list l contains a new np.array for each time-step of the game of life
# Calculate, how the entropy changes over time and visualize the output
# We recommend to use a plotly line-chart for visualization https://plotly.com/python/line-charts/